In [1]:
    '''
    
    
    Parameters
    ----------
    
    Returns
    -------
    '''

'\n\n\nParameters\n----------\n\nReturns\n-------\n'

In [2]:
'''
Created on Jun 24, 2018

@author: Mark Holton
'''

import sys


import pandas as pd
import numpy as np


print("start")

start


Comments are for code below
Reads a text file in and converts it to a dataframe. Due to the large size of the file a smaller file is created with hard coded columns for testing.

In [3]:
#read the file in and
textfile = "./ag_map_with_alpha.txt"
originalDF=pd.read_csv(textfile,
                        sep = "\t")
masterDF = originalDF.copy()
masterDF_short=masterDF.loc[0:200,['#SampleID',  
                                            'age_years',    
                                            'antibiotic_history',    
                                            'bmi_cat',  
                                            'depression_bipolar_schizophrenia', 
                                            'mental_illness_type_depression', 
                                            'smoking_frequency']]#.set_index('#SampleID', drop = False)
masterDF = masterDF_short.copy()  


/home/markholton/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def get_user_input_exclude():
    '''
    converts user_input_exclude file into user_input_excludeDF dataframe
    
    user_input_exclude is the file that contains the data needed to determine what samples to exclude. The file is a 
    tab seperated. The first line has the names of columns to look at for each filter condition. The second line is 
    what type of filter. Keep means that any row that satisfies the condition layed out below will be kept. Exclude 
    means rows that satisfy the condition will be filtered out. Third line is the filter comparison type. It can be <, 
    >, <=, >=, or ==. The forth (and following lines if == is chosen) form the end of the comparison. 
    
    EX. If one column of the file is
    age_years
    keep
    <
    50
    
    then the program will keep and rows where age_years is less than 50.

    Parameters
    ----------
    none
    
    Returns
    -------
    user_input_excludeDF
        dataframe that contains the information in user_input_exclude file
    '''
    user_input_exclude = 'exclude' # input("Filename for file with column and data to exclude: ")
    user_input_exclude = './%s.txt'%(user_input_exclude)
    user_input_excludeDF = pd.read_csv(user_input_exclude, sep = "\t")
    return user_input_excludeDF

In [5]:
def get_user_input_control():
    '''
    converts user_input_control file into user_input_controlDF dataframe
    
    user_input_control is the file that contains the data needed to determine what samples should be labeled control. 
    The file is tab seperated. The first line has the names of columns to look at for labeling. The following lines 
    contain strings. If the data for the coresponding column in the sample dataframe matches any of the previously 
    mentioned strings, the sample is labeled control in the case_control column.
    
    Parameters
    ----------
    none
    
    Returns
    -------
    user_input_controlDF
        dataframe that contains the information in user_input_control file
    '''
    user_input_control = 'control' # input("Filename for file with column and data to determine control: ")
    user_input_control = './%s.txt'%(user_input_control)
    user_input_controlDF = pd.read_csv(user_input_control, sep = "\t")
    return user_input_controlDF

In [6]:
def get_user_input_experiment():
    '''
    converts user_input_experiment file into user_input_experimentDF dataframe
    
    user_input_experiment is the file that contains the data needed to determine what samples should be labeled case. 
    The file is tab seperated. The first line has the names of columns to look at for labeling.The following lines 
    contain strings. If the data for the coresponding column in the sample dataframe matches any of the previously 
    mentioned strings, the sample is labeled case in the case_control column.
    
    Parameters
    ----------
    none
    
    Returns
    -------
    user_input_experimentDF
        dataframe that contains the information in user_input_experiment file
    '''
    user_input_experiment = 'experiment' # input("Filename for file with column and data to determine experimental: ")
    user_input_experiment = './%s.txt'%(user_input_experiment )
    user_input_experimentDF = pd.read_csv(user_input_experiment, sep = "\t")
    return user_input_experimentDF

In [7]:
def get_user_input_match():
    '''
    converts user_input_match file into user_input_matchDF dataframe
    
    user_input_match is the file that contains the data needed to determine what case and control samples should 
    be matched on. The file is tab seperated. The first line has the names of columns that samples will be matched on. 
    The second line contains what data type its coresponding column contains in the dataframe of samples. 
    It can be string or number. The third line contain different information depending on which column of the file 
    they are in. Columns where line 3 was string will be blank since stings are matched exactly. Columns where line 3 
    was number will be a number. This number is how far away sample data can be for the given column to still be matched. 
    
    EX. if the file text was
    bmi_cat age_years smoking_frequency
    string number string
    5
    
    Then the case and control samples would be matched to have the same bmi_cat and smoking_frequency. 
    Samples would also have to come from individuals within a five year distance in age to be matched.

    Parameters
    ----------
    none
    
    Returns
    -------
    user_input_matchDF
        dataframe that contains the information in user_input_match file
    '''
    user_input_match = 'match' # input("Filename for file with columns, type of value, and offset if needed to match samples: ")
    user_input_match= './%s.txt'%(user_input_match)
    user_input_matchDF = pd.read_csv(user_input_match, sep = "\t")
    return user_input_matchDF

In [8]:
user_input_excludeDF = get_user_input_exclude()
user_input_controlDF = get_user_input_control()
user_input_experimentDF = get_user_input_experiment()
user_input_matchDF = get_user_input_match()

Filters out unwanted rows based on values in chosen columns.

In [9]:
def exclude_samples(masterDF, inputDF):
    '''
    Filters out unwanted rows based on values in chosen columns.
    
    Parameters
    ----------
    masterDF : dataframe
        dataframe with all samples that needs to have samples filtered out
    
    inputDF : dataframe'
        dataframe with information to determine what samples to be filtered out
    
    Returns
    -------
    excludeDF : dataframe
        dataframe with samples filtered out based on outcome of conditional statements
    '''
    
    rows_num , columns_num = inputDF.shape
    print('rows  = %s, columns = %s'%(rows_num, columns_num) )
    r_counter = 0
    c_counter = 0

    while c_counter < columns_num:
        # column to look at for determining exclution 
        column_name = inputDF.columns[c_counter] 
        # data satifies the conditional statements below should the data be kept of excluded
        keep_if_match = ( inputDF.loc[0,column_name] == 'keep' )
        # type of filter can be <= < == > >= 
        type_of_filter =  inputDF.loc[1,column_name]
        masterDF = masterDF[ masterDF[column_name].isin( ['Unspecified'] ) == False ]
        #conditional statements
        if type_of_filter == '==':
            masterDF = masterDF[
                    masterDF[column_name].isin( inputDF[column_name].tolist() ) 
                    == 
                    ( keep_if_match ) ]
        if type_of_filter == '<=':
            masterDF = masterDF[
                    ( pd.to_numeric(masterDF[column_name]) <= float(inputDF.loc[2,column_name]) ) 
                    == 
                    ( keep_if_match ) ]
        if type_of_filter == '>=':
            masterDF = masterDF[
                    ( pd.to_numeric(masterDF[column_name]) >= float(inputDF.loc[2,column_name]) ) 
                    == 
                    ( keep_if_match ) ]
        if type_of_filter == '<':
            masterDF = masterDF[
                    ( pd.to_numeric(masterDF[column_name]) < float(inputDF.loc[2,column_name]) ) 
                    == 
                    ( keep_if_match ) ]
        if type_of_filter == '>':    
            masterDF = masterDF[
                    ( pd.to_numeric(masterDF[column_name]) > float(inputDF.loc[2,column_name]) ) 
                    == 
                    ( keep_if_match ) ]
        if type_of_filter == '> <':    
            masterDF = masterDF[
                    ( 
                    pd.to_numeric(masterDF[column_name]) > float(inputDF.loc[2,column_name]) 
                    &
                    pd.to_numeric(masterDF[column_name]) < float(inputDF.loc[3,column_name])
                    ) 
                    == 
                    ( keep_if_match ) ]
        if type_of_filter == '>= <=':    
            masterDF = masterDF[
                    ( 
                    pd.to_numeric(masterDF[column_name]) >= float(inputDF.loc[2,column_name]) 
                    &
                    pd.to_numeric(masterDF[column_name]) <= float(inputDF.loc[3,column_name])
                    )
                    == 
                    ( keep_if_match ) ]
        if type_of_filter == '> <=':    
            masterDF = masterDF[
                    ( 
                    pd.to_numeric(masterDF[column_name]) > float(inputDF.loc[2,column_name]) 
                    &
                    pd.to_numeric(masterDF[column_name]) <= float(inputDF.loc[3,column_name])
                    ) 
                    == 
                    ( keep_if_match ) ]
        if type_of_filter == '>= <':    
            masterDF = masterDF[
                    ( 
                    pd.to_numeric(masterDF[column_name]) >= float(inputDF.loc[2,column_name]) 
                    &
                    pd.to_numeric(masterDF[column_name]) < float(inputDF.loc[3,column_name])
                    ) 
                    == 
                    ( keep_if_match ) ]

        #increment counter  
        c_counter = c_counter + 1

    return masterDF.copy()    
    
    

In [10]:
excludeDF = exclude_samples(masterDF, user_input_excludeDF)


rows  = 3, columns = 2


Lables the samples based on if they are control or not

In [11]:
def filter_control_case(type_of_filter, column_name, masterDF, inputDF):
    ''' Lables the samples based on if they satisfy certain conditions
    
    Parameters
    ----------
    type_of_filter : str
        what way to compare inputDF values to masterDF
        can be <, >, ==, > <, <=, >=, <= >=, < >=, <= >
        == is the only one that can be used on string values
        
    column_name : str
        name of the column to look at for values. Only data in that column will be looked at. Must be the name of a 
        column in both masterDF and inputDF
        
    masterDF : panda dataframe
        dataframe of metadata for samples after exclutions are done
        
    inputDF : panda dataframe
        Can be user_input_control or user_input_experiment dataframes
        
    Returns
    -------
    masterDF 
        dataframe with column case_control labled to reflect if it satified the column_name and type_of_filter 
        conditional statements below
    '''

    
    # First checks to see if for the sample is already true for case_control. 
    if type_of_filter == '==':
        # Then check if for a column its value is in the list of values in the input dataframe.
        masterDF['case_control'] = np.where(masterDF['case_control'].isin([True]) 
                                            & 
                                            masterDF[column_name].isin( inputDF[column_name].tolist() ), 
                                            True, False)    
    # Then check if value in masterDF is "type_of_filter" corresponding value in inputDF
    if type_of_filter == '<=':
        masterDF['case_control'] = np.where(masterDF['case_control'].isin([True]) 
                                            & 
                                            ( pd.to_numeric(masterDF[column_name]) <= float(inputDF.loc[1,column_name]) ),
                                            True, False)           
    if type_of_filter == '>=':
        masterDF['case_control'] = np.where(masterDF['case_control'].isin([True]) 
                                            & 
                                            ( pd.to_numeric(masterDF[column_name]) >= float(inputDF.loc[1,column_name]) ),
                                            True, False)           
    if type_of_filter == '<': 
        masterDF['case_control'] = np.where(masterDF['case_control'].isin([True])                                        
                                            & 
                                            ( pd.to_numeric(masterDF[column_name]) < float(inputDF.loc[1,column_name]) ),
                                            True, False)           
    if type_of_filter == '>':    
        masterDF['case_control'] = np.where(masterDF['case_control'].isin([True]) 
                                            & 
                                            ( pd.to_numeric(masterDF[column_name]) > float(inputDF.loc[1,column_name]) ),
                                            True, False)      
    # Then check if value in masterDF is within the bounds (inclusively or not based on type_of_filter) of the numbers
    # in inputDF
    if type_of_filter == '> <':    
        masterDF['case_control'] = np.where(masterDF['case_control'].isin([True]) 
                                            & 
                                            ( pd.to_numeric(masterDF[column_name]) > float(inputDF.loc[1,column_name]) )
                                            &
                                            ( pd.to_numeric(masterDF[column_name]) < float(inputDF.loc[2,column_name]) ),
                                            True, False)           
    if type_of_filter == '>= <=':    
        masterDF['case_control'] = np.where(masterDF['case_control'].isin([True]) 
                                            & 
                                            ( pd.to_numeric(masterDF[column_name]) >= float(inputDF.loc[1,column_name]) )
                                            &
                                            ( pd.to_numeric(masterDF[column_name]) <= float(inputDF.loc[2,column_name]) ),
                                            True, False)          
    if type_of_filter == '> <=':    
        masterDF['case_control'] = np.where(masterDF['case_control'].isin([True]) 
                                            & 
                                            ( pd.to_numeric(masterDF[column_name]) > float(inputDF.loc[1,column_name]) )
                                            &
                                            ( pd.to_numeric(masterDF[column_name]) <= float(inputDF.loc[2,column_name]) ),
                                            True, False)
    if type_of_filter == '>= <':    
        masterDF['case_control'] = np.where(masterDF['case_control'].isin([True]) 
                                            & 
                                            ( pd.to_numeric(masterDF[column_name]) >= float(inputDF.loc[1,column_name]) )
                                            &
                                            ( pd.to_numeric(masterDF[column_name]) < float(inputDF.loc[2,column_name]) ),
                                            True, False)     
    
    return masterDF

In [12]:
def get_controlDF(masterDF, inputDF):
    '''
    
    
    Parameters
    ----------
    masterDF : dataframe
    
    inputDF : dataframe
    
    Returns
    -------
    controlDF : dataframe
    
    '''
    rows_num , columns_num = inputDF.shape
    print('rows  = %s, columns = %s'%(rows_num, columns_num) )
    r_counter = 0
    c_counter = 0
    #creates a new case_control column of the dataframe and sets all the values to false
    masterDF['case_control'] = True

    while c_counter < columns_num:
        column_name = inputDF.columns[c_counter]
        type_of_filter =  inputDF.loc[0,column_name]


        masterDF = filter_control_case(type_of_filter, column_name, masterDF, inputDF)


        #increment counter
        c_counter = c_counter + 1


    #replaces the true values created by the loop above to control
    masterDF['case_control'] = masterDF['case_control'].replace(True, 'control')   
    #replaces the false values created by the loop above to Unspecified
    masterDF['case_control'] = masterDF['case_control'].replace(False, 'Unspecified' )

    return masterDF.copy()

In [13]:
controlDF = get_controlDF(excludeDF.copy(), user_input_controlDF.copy())
#controlDF

rows  = 3, columns = 2


Lables the samples based on if they are cases or not

In [14]:
def get_caseDF(masterDF, inputDF):
    '''
    
    
    Parameters
    ----------
    masterDF : dataframe
    
    inputDF : dataframe
    
    Returns
    -------
    experDF : dataframe
    
    '''
    rows_num , columns_num = inputDF.shape
    print('rows  = %s, columns = %s'%(rows_num, columns_num) )
    r_counter = 0
    c_counter = 0

    #creates a new case_control column of the dataframe and sets all the values to false
    masterDF['case_control'] = True

    while c_counter < columns_num:
        column_name = inputDF.columns[c_counter]
        type_of_filter =  inputDF.loc[0,column_name]


        masterDF = filter_control_case(type_of_filter, column_name, masterDF, inputDF)


        #increment counter
        c_counter = c_counter + 1

    #replaces the true values created by the loop above to case
    masterDF['case_control'] = masterDF['case_control'].replace(True, 'case')   
    #replaces the false values created by the loop above to Unspecified
    masterDF['case_control'] = masterDF['case_control'].replace(False, 'Unspecified')

    return masterDF.copy()
    

In [15]:
caseDF = get_caseDF( excludeDF.copy(), user_input_experimentDF.copy() )


rows  = 4, columns = 2


In [16]:
def merge_caseDF_and_controlDF(caseDF, controlDF):
    '''
    Combines the two case_control columns into one column using pandas.DataFrame.where then makes this the master 
    dataframe's case_control column
    
    Parameters
    ----------
    caseDF : dataframe
        
        
    controlDF : dataframe
        
        
    Returns
    -------
    case_controlDF : dataframe
    
    '''
    #makes the case control column equal to is value in controlDF dataframe unless it is case in the experDF dataframe
    masterDF['case_control'] = caseDF['case_control'].where( 
                                                            caseDF['case_control'].isin(['case']), 
                                                            controlDF['case_control'])

    return masterDF.copy()


In [17]:
case_controlDF = merge_caseDF_and_controlDF(caseDF, controlDF)

Filters out data with match columns left unspecified

In [18]:
def filter_prep_for_matchDF(non_filteredDF, inputDF):
    '''
    
    
    Parameters
    ----------
    non_filteredDF : dataframe
    
    inputDF : dataframe
    
    Returns
    -------
    match_filteredDF : dataframe
    '''
    rows_num , columns_num = inputDF.shape
    print('rows  = %s, columns = %s'%(rows_num, columns_num) )
    r_counter = 0
    c_counter = 0
    while c_counter < columns_num:
            column_name = inputDF.columns[c_counter]
            # only keep samples where for the chosen columns no values equal Unspecified
            non_filteredDF = non_filteredDF.where(non_filteredDF[column_name] != 'Unspecified')
            c_counter = c_counter + 1
    non_filteredDF = non_filteredDF.dropna()
    #masterDF = masterDF.reset_index(drop=True)


    return non_filteredDF.copy()


In [19]:
      
match_filteredDF = filter_prep_for_matchDF(case_controlDF.copy(), user_input_matchDF.copy())


rows  = 2, columns = 3


Sets the matched to column of master_frame to the case sample (case_row) ID for the control samples still left in match_frame

In [20]:
def match_to_master(master_frame, match_frame, case_index, case_row):
    '''
    Labels control samples with what case sample they match to in the matched to column
    
    Parameters
    ----------
    master_frame : dataframe
        what will have its matched to column changed
    match_frame : dataframe
        contains the samples that match to case_row
    case_index : integer
        index of case_row in master_frame
    case_row : series?
        case sample that matches to samples in match_frame

    Returns
    -------
    master_frame
        output is the updated master_frame with matched to column reflecting the case id number it matches to for controls
        and cases are labeled 1 in matched to column
    '''
    for index, row in match_frame.iterrows():
        #print(master_frame.at[index, 'matched to'] )
        master_frame.at[index, 'matched to'] = str(case_row['#SampleID']) 
        #sets the case sample matched to value to 1 in master_frame
        master_frame.at[case_index, 'matched to'] = '1'#best match place holder'
    
    
    return master_frame

Matches samples labled experiment to control samples

In [21]:
def match_samples(masterDF, inputDF):
    '''
    
    
    Parameters
    ----------
    masterDF : dataframe
    
    inputDF : dataframe
    
    Returns
    -------
    masterDF : dataframe
    
    '''
    
    rows_num , columns_num = inputDF.shape
    print('rows  = %s, columns = %s'%(rows_num, columns_num) )
    # creates row and column counters
    r_counter = 0
    c_counter = 0

    case_for_matchDF = masterDF[masterDF['case_control'].isin(['case'])]
    # creates column to show matches. since it will contain the sample number it was matched too the null value will be 0
    masterDF['matched to'] = '0'

    matchDF = masterDF.copy()

    # loops though case samples and matches them to controls
    for index, row in case_for_matchDF.iterrows():
        # set matchDF to be only the samples of masterDF that are control samples
        matchDF = masterDF.copy()
        matchDF = matchDF[matchDF['case_control'].isin(['control'])]

        # loop though input columns to determine matches
        #print(index)
        c_counter = 0
        while c_counter < columns_num:
            column_name = inputDF.columns[c_counter]
            # get the type of data for the given column. This determine how a match is determined
            datatype = inputDF.iloc[0,c_counter]
            if datatype == 'string':
                # filters controls if the strings for the control and case don't match
                matchDF = matchDF[matchDF[column_name].isin([ row[column_name] ]) ]
            if datatype == 'number':
                # filters controls based on if the value in the control is not within a given distance form the case
                matchDF = matchDF[
                                    ( pd.to_numeric(matchDF[column_name]) >= ( int(float(row[column_name])) - int(float(inputDF.iloc[1,c_counter])) ) ) 
                                    &
                                    ( pd.to_numeric(matchDF[column_name]) <= ( int(float(row[column_name])) + int(float(inputDF.iloc[1,c_counter])) ) )
                                   ] 
            # increment loop counter
            c_counter = c_counter + 1

        # sets the matched to column of masterDF to the case sample ID for the control samples still left in matchDF
        masterDF = match_to_master(masterDF, matchDF, index, row)

    
    return masterDF.copy()

In [22]:
masterDF = match_samples( match_filteredDF.copy(), user_input_matchDF.copy() )

rows  = 2, columns = 3


In [23]:
masterDF['matched to'].unique()

array(['0', '10317.000029448', '1'], dtype=object)

In [24]:
masterDF


,#SampleID,age_years,antibiotic_history,bmi_cat,depression_bipolar_schizophrenia,mental_illness_type_depression,smoking_frequency,case_control,matched to
1,10317.000049761,52,I have not taken antibiotics in the past year.,Overweight,Unspecified,Unspecified,Never,Unspecified,0
4,10317.000030322,43.0,I have not taken antibiotics in the past year.,Normal,I do not have this condition,Unspecified,Never,control,10317.000029448
6,10317.000028857,56.0,I have not taken antibiotics in the past year.,Overweight,I do not have this condition,Unspecified,Never,Unspecified,0
8,10317.000038189,39.0,I have not taken antibiotics in the past year.,Overweight,Unspecified,Unspecified,Never,control,0
10,10317.000036487,46.0,I have not taken antibiotics in the past year.,Obese,I do not have this condition,Unspecified,Never,Unspecified,0
11,10317.000030166,58.0,I have not taken antibiotics in the past year.,Overweight,I do not have this condition,Unspecified,Never,Unspecified,0
12,10317.000023295,56.0,I have not taken antibiotics in the past year.,Normal,I do not have this condition,Unspecified,Never,Unspecified,0
13,10317.000001589,58.0,I have not taken antibiotics in the past year.,Normal,Unspecified,Unspecified,Never,Unspecified,0
16,10317.000021163,56.0,I have not taken antibiotics in the past year.,Normal,I do not have this condition,Unspecified,Never,Unspecified,0
20,10317.000029448,44.0,I have not taken antibiotics in the past year.,Normal,Self-diagnosed,Unspecified,Never,case,1
